## Homework 1   
-----
__Author: Turakulov A.__   
__Group: BASB212__  
The website will be parsed according to the requirements https://www.kinopoisk.ru/lists/movies/top250/    
Then the Mann-Whitney U test and Bootstrap will be executed to check the hypotheses

In [47]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import selenium.webdriver as webdriver
import os
import warnings
warnings.filterwarnings("ignore")

Here selenium is used in order to imitate human-like requests to the website   
If such an imitation is not used, it is impossible to loop and execute queries

In [48]:
working_dir = os.getcwd()
webdriver_path = os.path.join(working_dir, "chromedriver")  # proper version https://chromedriver.chromium.org/
# webdriver settings
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36")

os.environ["webdriver.chrome.driver"] = webdriver_path  # 'webdriver' executable needs to be in PATH

data =[]

for i in range(1,6):
    driver=webdriver.Chrome(executable_path=webdriver_path, chrome_options=chrome_options)
    driver.get(f"https://www.kinopoisk.ru/lists/movies/top250/?page={str(i)}")
    soup=BeautifulSoup(driver.page_source,'lxml')
    films=soup.findAll('div',class_='styles_root__ti07r')
    for film in films:
        name=film.find('a',class_='base-movie-main-info_link__YwtP1').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text#.find('span', class_='desktop-list-main-info_secondaryText__M_aus')
        name_rus=film.find('div',class_='styles_content__nT2IG').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
        genre=film.find('div',class_='styles_content__nT2IG').findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text
        rating_new=film.find('div',class_='styles_rating__ni2L0 styles_root___s7Tg styles_rootMd__ZvdRj styles_rootPositive__PIwO2').text
        rating_old=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg').text
        views=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskCount__2_VPQ').text

        data.append([name,name_rus,genre,rating_new,rating_old,views])

In [74]:
header=['name','name_rus','genre','rating_new','rating_old','views']

In [120]:
df=pd.DataFrame(data,columns=header)
df

,name,name_rus,genre,rating_new,rating_old,views
0,"The Green Mile, 1999, 189 мин.",Зеленая миля,США • драма Режиссёр: Фрэнк Дарабонт,9.1,9.2,121 672
1,"Schindler's List, 1993, 195 мин.",Список Шиндлера,США • драма Режиссёр: Стивен Спилберг,8.8,9.1,76 487
2,"The Shawshank Redemption, 1994, 142 мин.",Побег из Шоушенка,США • драма Режиссёр: Фрэнк Дарабонт,9.1,9.0,79 053
3,"Forrest Gump, 1994, 142 мин.",Форрест Гамп,США • драма Режиссёр: Роберт Земекис,8.9,8.9,105 297
4,"The Lord of the Rings: The Return of the King,...",Властелин колец: Возвращение короля,Новая Зеландия • фэнтези Режиссёр: Питер Джексон,8.7,8.9,50 777
...,...,...,...,...,...,...
245,"1987, 98 мин.",Человек с бульвара Капуцинов,СССР • вестерн Режиссёр: Алла Сурикова,8.0,8.1,15 937
246,"Goodfellas, 1990, 140 мин.",Славные парни,США • драма Режиссёр: Мартин Скорсезе,8.1,8.1,27 663
247,"Awakenings, 1990, 121 мин.",Пробуждение,США • драма Режиссёр: Пенни Маршалл,8.4,8.1,12 652
248,"Rush, 2013, 123 мин.",Гонка,Великобритания • спорт Режиссёр: Рон Ховард,8.1,8.1,26 801


In [121]:
len(df['name_rus'].unique())

250

Now we have to split column into several columns in order to get such an information as country, genre, director and year of movie

In [122]:
df[['country', 'genre_director']] = df['genre'].str.split(' • ', 1, expand=True)
df.drop(columns = ['genre'], inplace = True)
df[['genre', 'director']] = df['genre_director'].str.split('Режиссёр:', 1, expand=True)
df.drop(columns = ['genre_director'], inplace = True)
df[['country','genre', 'director']] = df[['country','genre', 'director']].apply(lambda x: x.str.strip())
arr_year = [int(el[-2]) for el in df['name'].str.split(', ', 4)]
df['year'] = arr_year
df

,name,name_rus,rating_new,rating_old,views,country,genre,director,year
0,"The Green Mile, 1999, 189 мин.",Зеленая миля,9.1,9.2,121 672,США,драма,Фрэнк Дарабонт,1999
1,"Schindler's List, 1993, 195 мин.",Список Шиндлера,8.8,9.1,76 487,США,драма,Стивен Спилберг,1993
2,"The Shawshank Redemption, 1994, 142 мин.",Побег из Шоушенка,9.1,9.0,79 053,США,драма,Фрэнк Дарабонт,1994
3,"Forrest Gump, 1994, 142 мин.",Форрест Гамп,8.9,8.9,105 297,США,драма,Роберт Земекис,1994
4,"The Lord of the Rings: The Return of the King,...",Властелин колец: Возвращение короля,8.7,8.9,50 777,Новая Зеландия,фэнтези,Питер Джексон,2003
...,...,...,...,...,...,...,...,...,...
245,"1987, 98 мин.",Человек с бульвара Капуцинов,8.0,8.1,15 937,СССР,вестерн,Алла Сурикова,1987
246,"Goodfellas, 1990, 140 мин.",Славные парни,8.1,8.1,27 663,США,драма,Мартин Скорсезе,1990
247,"Awakenings, 1990, 121 мин.",Пробуждение,8.4,8.1,12 652,США,драма,Пенни Маршалл,1990
248,"Rush, 2013, 123 мин.",Гонка,8.1,8.1,26 801,Великобритания,спорт,Рон Ховард,2013


In [83]:
df['country'].unique()

array(['США', 'Новая Зеландия', 'Франция', 'СССР', 'Япония', 'Германия',
       'Великобритания', 'Испания', 'Австралия', 'Италия', 'Россия',
       'Ирландия', 'Швеция', 'Беларусь', 'Дания', 'Корея Южная'],
      dtype=object)

In [84]:
df['director'].unique()

array(['Фрэнк Дарабонт', 'Стивен Спилберг', 'Роберт Земекис',
       'Питер Джексон', 'Кристофер Нолан', 'Квентин Тарантино',
       'Оливье Накаш', 'Ли Анкрич', 'Леонид Гайдай', 'Роджер Аллерс',
       'Хаяо Миядзаки', 'Дэвид Финчер', 'Ридли Скотт', 'Эндрю Стэнтон',
       'Эндрю Адамсон', 'Игорь Масленников', 'Александр Серый',
       'Владимир Меньшов', 'Томас Ян', 'Мартин Скорсезе',
       'Давид Черкасский', 'Станислав Ростоцкий', 'Гор Вербински',
       'Гай Ричи', 'Леонид Быков', 'Сергей Бондарчук', 'Серхио Паблос',
       'Мэл Гибсон', 'Роберто Бениньи', 'Тейт Тейлор', 'Крис Коламбус',
       'Алексей Балабанов', 'Альфонсо Куарон', 'Джеймс Кэмерон',
       'Фрэнсис Форд Коппола', 'Люк Бессон', 'Лана Вачовски',
       'Владимир Бортко', 'Владимир Роговой', 'Эльдар Рязанов',
       'Мартин Брест', 'Питер Фаррелли', 'Лассе Халльстрём',
       'Роман Полански', 'Дин ДеБлуа', 'Байрон Ховард', 'Сидни Люмет',
       'Макото Синкай', 'Юрий Чулюкин', 'Дэмьен Шазелл', 'Брэд Бёрд',
      

In [82]:
df['genre'].unique()

array(['драма', 'фэнтези', 'фантастика', 'криминал', 'мультфильм',
       'комедия', 'аниме', 'триллер', 'история', 'детектив', 'мелодрама',
       'военный', 'боевик', 'биография', 'вестерн', 'приключения',
       'мюзикл', 'ужасы', 'спорт'], dtype=object)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        250 non-null    object
 1   name_rus    250 non-null    object
 2   rating_new  250 non-null    object
 3   rating_old  250 non-null    object
 4   views       250 non-null    object
 5   country     250 non-null    object
 6   genre       250 non-null    object
 7   director    250 non-null    object
dtypes: object(8)
memory usage: 15.8+ KB


Here it can be seen that there are no null rows in the dataframe

## Your hometask is to check the following hypothesis:

__H0__ - there is no statistical difference between rating_new and rating_old

__Ha__ - the difference between rating_new and rating_old exists

using statistics Mann-Whitney and Boostrap. Compare all the ratings, ratings of movies of a similar genre/decade of creating/country of origin

---------
Mann-Whitney U test used for comparing differences between two independent groups. It tests the hypothesis that if the two groups come from same population or have the same medians. It does not assume any specific distribution (such as normal distribution of samples) for calculating test statistics and p values.

First of all we have to compare all the ratings. 

In [137]:
import scipy.stats as stats

#perform the Mann-Whitney U test
statistic, p_value = stats.mannwhitneyu(list(map(float, df['rating_new'].to_list()))
                    ,list(map(float, df['rating_old'].to_list()))
                    ,alternative='two-sided'
                   )
print(f"Statistic: {statistic}")
print('P-value: {:.15f}'.format(p_value))
if p_value > 0.05: 
    print('P-value is greater than 0.05. We fail to reject the null hypothesis')
else: 
    print('P-value is less than 0.05. We succeed to reject the null hypothesis')


Statistic: 22502.0
P-value: 0.000000042112947
P-value is less than 0.05. We succeed to reject the null hypothesis


Then we can check hypothesis comparing movies with similar genre

In [143]:
for curr_genre in df['genre'].unique():
    print(f'Performing Mann-Whitney test for genre: {curr_genre}')
    try:
        statistic, p_value = stats.mannwhitneyu(list(map(float, df[df['genre'] == curr_genre]['rating_new'].to_list()))
                            ,list(map(float, df[df['genre'] == curr_genre]['rating_old'].to_list()))
                            ,alternative='two-sided'
                           )
        print(f"Statistic: {statistic}")
        print('P-value: {:.15f}'.format(p_value))
        if p_value > 0.05: 
            print('P-value is greater than 0.05. We fail to reject the null hypothesis')
        else: 
            print('P-value is less than 0.05. We succeed to reject the null hypothesis')
    except Exception as msg:
        print(f'Skipping genre "{curr_genre}" because: ' + str(msg))
        
    print('------')

Performing Mann-Whitney test for genre: драма
Statistic: 1627.0
P-value: 0.135562864425177
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for genre: фэнтези
Statistic: 79.5
P-value: 0.003120030694137
P-value is less than 0.05. We succeed to reject the null hypothesis
------
Performing Mann-Whitney test for genre: фантастика
Statistic: 207.5
P-value: 0.016120940133222
P-value is less than 0.05. We succeed to reject the null hypothesis
------
Performing Mann-Whitney test for genre: криминал
Statistic: 39.0
P-value: 0.928562236307366
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for genre: мультфильм
Statistic: 477.5
P-value: 0.000011027670676
P-value is less than 0.05. We succeed to reject the null hypothesis
------
Performing Mann-Whitney test for genre: комедия
Statistic: 63.0
P-value: 0.893985512806365
P-value is greater than 0.05. We fail to reject the null hypothesis
--

Then we can check hypothesis comparing movies created in specific country

In [144]:
for curr_country in df['country'].unique():
    print(f'Performing Mann-Whitney test for country: {curr_country}')
    try:
        statistic, p_value = stats.mannwhitneyu(list(map(float, df[df['country'] == curr_country]['rating_new'].to_list()))
                            ,list(map(float, df[df['country'] == curr_country]['rating_old'].to_list()))
                            ,alternative='two-sided'
                           )
        print(f"Statistic: {statistic}")
        print('P-value: {:.15f}'.format(p_value))
        if p_value > 0.05: 
            print('P-value is greater than 0.05. We fail to reject the null hypothesis')
        else: 
            print('P-value is less than 0.05. We succeed to reject the null hypothesis')
    except Exception as msg:
        print(f'Skipping country "{curr_country}" because: ' + str(msg))
        
    print('------')

Performing Mann-Whitney test for country: США
Statistic: 6818.5
P-value: 0.000008207609282
P-value is less than 0.05. We succeed to reject the null hypothesis
------
Performing Mann-Whitney test for country: Новая Зеландия
Statistic: 9.0
P-value: 0.171223027064911
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for country: Франция
Statistic: 22.0
P-value: 0.313391406837020
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for country: СССР
Statistic: 872.0
P-value: 0.648968427492868
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for country: Япония
Statistic: 42.5
P-value: 0.582154665424915
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for country: Германия
Statistic: 3.5
P-value: 0.239639046607999
P-value is greater than 0.05. We fail to reject the null hypothesi

Finally, we check hypothesis comparing movies released in certain decade

In [166]:
import numpy as np
years = np.sort(df['year'].unique())
decades = []
for each in years:
    decade = int(np.floor(each / 10) * 10)
    decades.append(decade)

print("All years: ",years)
print("All decades: ",sorted(set(decades))) 

for curr_decade in sorted(set(decades)):
    print(f'Performing Mann-Whitney test for decade: {curr_decade}')
    try:
        statistic, p_value = stats.mannwhitneyu(list(map(float, df[(df['year'] >= curr_decade)&(df['year'] < curr_decade+10)]['rating_new'].to_list()))
                            ,list(map(float, df[(df['year'] >= curr_decade)&(df['year'] < curr_decade+10)]['rating_old'].to_list()))
                            ,alternative='two-sided'
                           )
        print(f"Statistic: {statistic}")
        print('P-value: {:.15f}'.format(p_value))
        if p_value > 0.05: 
            print('P-value is greater than 0.05. We fail to reject the null hypothesis')
        else: 
            print('P-value is less than 0.05. We succeed to reject the null hypothesis')
    except Exception as msg:
        print(f'Skipping decade "{curr_decade}" because: ' + str(msg))
        
    print('------')

All years:  [1939 1955 1956 1957 1959 1961 1962 1964 1965 1966 1968 1969 1970 1971
 1972 1973 1974 1975 1976 1977 1979 1980 1981 1982 1983 1984 1985 1986
 1987 1988 1989 1990 1991 1992 1993 1994 1995 1997 1998 1999 2000 2001
 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020 2021]
All decades:  [1930, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
Performing Mann-Whitney test for decade: 1930
Statistic: 1.0
P-value: 1.000000000000000
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for decade: 1950
Statistic: 24.5
P-value: 0.322748053511122
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for decade: 1960
Statistic: 70.0
P-value: 0.548601412617703
P-value is greater than 0.05. We fail to reject the null hypothesis
------
Performing Mann-Whitney test for decade: 1970
Statistic: 255.0
P-value: 0.841258901640309
P-value is greater tha

In order to verify the results of Mann-Whitney test we can perform Bootstrap test.   
Mean was chosen as a metric for this test

In [171]:
from bstrap import bootstrap

old_stats, new_stats, p_value = bootstrap(np.mean
                                          , df['rating_old'].astype('float') 
                                          , df['rating_new'].astype('float')
                                          , nbr_runs=1000)
old_stats, new_stats, p_value

({'avg_metric': 8.316137599999998,
  'metric_ci_lb': 8.294799999999993,
  'metric_ci_ub': 8.339599999999995},
 {'avg_metric': 8.213263599999994,
  'metric_ci_lb': 8.186800000000003,
  'metric_ci_ub': 8.240820000000005},
 0.0)

The result same as Mann-Whitney test

In [173]:
old_stats, new_stats, p_value = bootstrap(np.mean
                                          , df[(df['year'] >= 1990)&(df['year'] < 1990+10)]['rating_old'].astype('float') 
                                          , df[(df['year'] >= 1990)&(df['year'] < 1990+10)]['rating_new'].astype('float')
                                          , nbr_runs=1000)
old_stats, new_stats, p_value

({'avg_metric': 8.391027659574469,
  'metric_ci_lb': 8.329787234042554,
  'metric_ci_ub': 8.457446808510639},
 {'avg_metric': 8.333942553191491,
  'metric_ci_lb': 8.27021276595745,
  'metric_ci_ub': 8.400000000000002},
 0.333)

We fail to reject the null hypothesis while comparing films from the 1990s.
There is a statistical difference between old and new ratings. The results are the same.